


# Assignment 5 on Natural Language Processing

## Date : 3rd Nov, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to explore various language models specifically LSTM based and transformer. We will explore how the size of the model effects the sequence generated. We will see both character based and word based models.



Please submit with outputs. Submissions without predicted outputs will be penalized.

# Word Based LSTM model

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras

Do basic pre processing which includes lowering etc
Check the dataset and apply suitable preprocessing.

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Load the data and preprocess data and store corpus in raw_text
dataset = open(r"/content/drive/My Drive/CS60075-NLP/NLP Assignment-5/corpus.txt","r+")
corpus = []
for line in dataset:
  corpus.append(line)

raw_text = "".join(corpus)


In [4]:
raw_text = raw_text.lower()

In [5]:
# initializing punctuations string  
punc = '''!()-[]{};:'"”\, <>./?@#$%^&*_~'''

word_text = raw_text 
for elem in word_text:  
    if elem in punc:  
        word_text = word_text.replace(elem, " ")


In [6]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = '<OOV>'
embedding_dim = 100
padding_type='post'
trunc_type='post'


In [7]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts([word_text])
word_index = tokenizer.word_index

In [8]:
seq_length = 50
tokens = tokenizer.texts_to_sequences([word_text])[0]

In [9]:
print(word_index)

{'<OOV>': 1, 'the': 2, 'and': 3, 'to': 4, 'a': 5, 'she': 6, 'of': 7, 'it': 8, 'said': 9, 'alice': 10, 'in': 11, 'was': 12, 'you': 13, 'i': 14, 'that': 15, 'as': 16, 'her': 17, 'at': 18, 'on': 19, 'with': 20, 'had': 21, 'all': 22, 'be': 23, 'for': 24, 'so': 25, 'very': 26, 'not': 27, 'this': 28, 'little': 29, 'but': 30, '“i': 31, 'they': 32, 'out': 33, 'he': 34, 'down': 35, 'what': 36, 'up': 37, 'is': 38, 'one': 39, 'his': 40, 'about': 41, 'were': 42, 'like': 43, 'went': 44, 'herself': 45, 'them': 46, 'again': 47, 'know': 48, 'then': 49, 'would': 50, 'could': 51, 'have': 52, 'no': 53, 'thought': 54, 'when': 55, 'if': 56, 'do': 57, 'time': 58, 'or': 59, 'there': 60, 'me': 61, 'queen': 62, 'into': 63, 'see': 64, 'off': 65, 'king': 66, 'your': 67, 'did': 68, 'began': 69, 'its': 70, 'by': 71, 'an': 72, 'my': 73, 'mock': 74, 'turtle': 75, '“and': 76, 'quite': 77, 'hatter': 78, 'gryphon': 79, 'way': 80, 'who': 81, 'don’t': 82, 'their': 83, 'think': 84, 'much': 85, 'say': 86, 'some': 87, 'thin

In [12]:
dataX = []
dataY = []

for i in range(0, len(tokens) - seq_length-1 , 1):
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  if seq_out==1: #Skip samples where target word is OOV
    continue
    
  dataX.append(seq_in)
  dataY.append(seq_out)
 
N = len(dataX)
print ("Total training data size: ", N)

Total training data size:  26524


In [13]:
X = numpy.array(dataX)

# one hot encode the output variable
y = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [14]:
# with embedding
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    keras.layers.Bidirectional(keras.layers.LSTM(128)),
    keras.layers.Dense(vocab_size, activation='softmax')
])
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               234496    
_________________________________________________________________
dense (Dense)                (None, 3000)              771000    
Total params: 1,305,496
Trainable params: 1,305,496
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Use validation split of 0.2 while training
model.fit(X, y, epochs=25, batch_size=128, validation_split=0.2) 

Epoch 1/25
166/166 [==============================] - 3s 21ms/step - loss: 6.5162 - accuracy: 0.0541 - val_loss: 6.3682 - val_accuracy: 0.0831
Epoch 2/25
166/166 [==============================] - 3s 16ms/step - loss: 6.0899 - accuracy: 0.0555 - val_loss: 6.3320 - val_accuracy: 0.0865
Epoch 3/25
166/166 [==============================] - 3s 16ms/step - loss: 5.9519 - accuracy: 0.0640 - val_loss: 6.3233 - val_accuracy: 0.0937
Epoch 4/25
166/166 [==============================] - 3s 16ms/step - loss: 5.8100 - accuracy: 0.0736 - val_loss: 6.2756 - val_accuracy: 0.0944
Epoch 5/25
166/166 [==============================] - 3s 16ms/step - loss: 5.6305 - accuracy: 0.0904 - val_loss: 6.2208 - val_accuracy: 0.1025
Epoch 6/25
166/166 [==============================] - 3s 16ms/step - loss: 5.4382 - accuracy: 0.1034 - val_loss: 6.1844 - val_accuracy: 0.1050
Epoch 7/25
166/166 [==============================] - 3s 16ms/step - loss: 5.2459 - accuracy: 0.1186 - val_loss: 6.1543 - val_accuracy: 0.1120

In [16]:
reverse_word_map = {value:key for (key,value) in word_index.items()}   

print(reverse_word_map)

{1: '<OOV>', 2: 'the', 3: 'and', 4: 'to', 5: 'a', 6: 'she', 7: 'of', 8: 'it', 9: 'said', 10: 'alice', 11: 'in', 12: 'was', 13: 'you', 14: 'i', 15: 'that', 16: 'as', 17: 'her', 18: 'at', 19: 'on', 20: 'with', 21: 'had', 22: 'all', 23: 'be', 24: 'for', 25: 'so', 26: 'very', 27: 'not', 28: 'this', 29: 'little', 30: 'but', 31: '“i', 32: 'they', 33: 'out', 34: 'he', 35: 'down', 36: 'what', 37: 'up', 38: 'is', 39: 'one', 40: 'his', 41: 'about', 42: 'were', 43: 'like', 44: 'went', 45: 'herself', 46: 'them', 47: 'again', 48: 'know', 49: 'then', 50: 'would', 51: 'could', 52: 'have', 53: 'no', 54: 'thought', 55: 'when', 56: 'if', 57: 'do', 58: 'time', 59: 'or', 60: 'there', 61: 'me', 62: 'queen', 63: 'into', 64: 'see', 65: 'off', 66: 'king', 67: 'your', 68: 'did', 69: 'began', 70: 'its', 71: 'by', 72: 'an', 73: 'my', 74: 'mock', 75: 'turtle', 76: '“and', 77: 'quite', 78: 'hatter', 79: 'gryphon', 80: 'way', 81: 'who', 82: 'don’t', 83: 'their', 84: 'think', 85: 'much', 86: 'say', 87: 'some', 88: '

In [17]:
# Complete the code to return next n words greedily
def next_tokens(input_str, n):
	print ("Seed: \n", input_str)
	input_text = input_str.lower()
	encoded_list = []
	final_string = input_text
	encoded_list = tokenizer.texts_to_sequences([input_text])[0]
	for i in range(n):
		encoded = numpy.reshape(encoded_list,(1,len(encoded_list),1))
		prediction = model.predict(encoded, verbose=0)
		pred_index = numpy.argmax(prediction)
		word = reverse_word_map[pred_index]
		final_string = final_string + " "+word
		encoded_list.append(pred_index)
		encoded_list = encoded_list[1:]
			# get next word index. Use reverse_word_map to get the word
	return final_string


In [18]:

start = numpy.random.randint(0, len(dataX)-1)

pattern = dataX[start]

input_str = ' '.join([reverse_word_map[value] for value in pattern])

print(next_tokens( input_str , 10))

Seed: 
 had taken his watch out of his pocket and was looking at it uneasily shaking it every now and then and holding it to his ear alice considered a little and then said “the fourth “two days wrong sighed the hatter “i told you butter wouldn’t suit the works he
had taken his watch out of his pocket and was looking at it uneasily shaking it every now and then and holding it to his ear alice considered a little and then said “the fourth “two days wrong sighed the hatter “i told you butter wouldn’t suit the works he added in a little many croquet with the three gardeners


In [20]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

# Use first 50 tokens from given input_str as input.(Use tokenizer to split to take first 50)
processed_input_str = input_str.lower() 

word_list = processed_input_str.split(" ")

input = ' '.join(word_list[:50])
print(next_tokens( input, 10))                 #edited

Seed: 
 the boy laughed at the fright he had caused. this time, the villagers left angrily. the third day, as the boy went up the small hill, he suddenly saw a wolf attacking his sheep. he cried as hard as he could, “wolf! wolf! wolf!”, but not  a single villager
the boy laughed at the fright he had caused. this time, the villagers left angrily. the third day, as the boy went up the small hill, he suddenly saw a wolf attacking his sheep. he cried as hard as he could, “wolf! wolf! wolf!”, but not  a single villager at the dormouse indignantly and she could not join the


# Character based LSTM Model 1

In [21]:
# User the preprocess data and create raw_text
punctu = '''!()-[]{};:\ <>/?@#$%^&*_~'''
char_text = raw_text.lower()
for elem in word_text:  
    if elem in punctu:  
        char_text = char_text.replace(elem, " ")
# create mapping of unique characters to integers
chars = sorted(list(set(char_text)))

char_to_int = dict((c, i) for i, c in enumerate(chars))

In [22]:
# Print the total characters and character vacob size

n_chars = len(char_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  142477
Total Vocab:  46


In [23]:
'''
Prepare dataset where the input is sequence of 100 characters and target is next character.
'''
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    # Write code here
    seq_in = char_text[i:i+seq_length]
    seq_out = char_text[i+seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  142377


In [24]:
len(dataX)

142377

In [25]:
# reshape X to be [samples, time steps, features]

X = numpy.reshape(dataX,(len(dataX),seq_length,1))

# one hot encode the output variable
dataY = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [26]:
embedding_dim =100
max_length =100

In [27]:
model = Sequential()
model.add(keras.layers.Embedding(n_vocab, embedding_dim, input_length=max_length))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          4600      
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 46)                11822     
Total params: 381,990
Trainable params: 381,990
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.fit(X, y, epochs=20, batch_size=128)

Epoch 1/20
1113/1113 [==============================] - 20s 18ms/step - loss: 2.2311
Epoch 2/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.7279
Epoch 3/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.5247
Epoch 4/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.4011
Epoch 5/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.3196
Epoch 6/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.2555
Epoch 7/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.2061
Epoch 8/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.1589
Epoch 9/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.1197
Epoch 10/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.0845
Epoch 11/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.0522
Epoch 12/20
1113/1113 [==============================] - 20s 18

In [29]:
#implement mapping of integer to character
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [30]:
'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	print("seed : ",pattern)
	print("\n")
	final_string = ""
	pattern = pattern.lower()
	pattern = [char_to_int[char] for char in pattern]
	for i in range(x):
		# Complete Code
		x = numpy.reshape(pattern,(1,len(pattern),1))
		prediction = model.predict(x, verbose=0)
		index = numpy.argmax(prediction[0])
		result = int_to_char[index]
		final_string += ''+result
		pattern.append(index)
		pattern = pattern[1:len(pattern)]	
	return final_string

In [31]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print(predict_next_100_chars(input_str,200))  # prediction is printed after the line gap 

seed :  ow us, with the lobsters, out to sea!” but the snail replied “too far, too far!” and gave a look ask


ing at all the three gardeners at the three gardeners, all the trees had a remark half happened, and shouted at alice as she could not stand at the moment she was so much a right so much at the thing 


In [32]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.

 
print(predict_next_100_chars(input_str[:100],200))  # prediction is printed after the line gap 

seed :  The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, a


nd she was now about the time they were all crowded all the time when she was a little shriek of the sort of things her head down and a long talking to see it was the words all crowded round her and t


## Character based LSTM Model 2


In [33]:
model1 = Sequential()
model1.add(keras.layers.Embedding(n_vocab, embedding_dim, input_length=max_length))
model1.add(LSTM(256, input_shape=(X.shape[1], embedding_dim),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')

In [34]:
model1.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
2225/2225 [==============================] - 48s 22ms/step - loss: 2.0936
Epoch 2/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.5642
Epoch 3/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.3897
Epoch 4/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.2918
Epoch 5/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.2191
Epoch 6/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.1679
Epoch 7/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.1243
Epoch 8/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.0896
Epoch 9/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.0550
Epoch 10/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.0293
Epoch 11/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.0043
Epoch 12/20
2225/2225 [==============================] - 49s 22

In [35]:
# Generate the sequence similar to above methods

'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	print("seed : ",pattern)
	print("\n")
	final_string = ""
	pattern = pattern.lower()
	pattern = [char_to_int[char] for char in pattern]
	for i in range(x):
		# Complete Code
		x = numpy.reshape(pattern,(1,len(pattern),1))
		prediction = model1.predict(x, verbose=0)
		index = numpy.argmax(prediction[0])
		result = int_to_char[index]
		final_string += ''+result
		pattern.append(index)
		pattern = pattern[1:len(pattern)]	
	return final_string

In [36]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print(predict_next_100_chars(input_str,200))

seed :  ” alice remarked.
“oh, you can’t help that,” said the cat: “we’re all mad here.
i’m mad.
you’re mad.


” “i can’t help it to see it is it?” said the caterpillar.
“well, it must be a little bill, i shall be a little birds!
oh dear, what a moment that i should like to be a book of the sea,” said the cate


In [37]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.

 
print(predict_next_100_chars(input_str[:100],200))     # prediction is printed after the line gap 

seed :  The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, a


nd then alice could not tell when they were nearly without a moment to be a long said, and then the mock turtle in a great hurry to the conversation the mock turtle, and then alice could not tell when


# Questions

**Question:** What are your observations based on the model(all) outputs on train data(in domain) vs unseen data(out of domain) ?

**Answer:**  


*   The relevance in context between input and generated text is high for the in domain / train data than on the unseen data. 
The generation for unseen data seems contextually irrelevant as model keeps predicting outputs based on the train data.
*   The characters from the input text such as alice, caterpillar,  show up in the generated text even though they were not present in input texts.
*   Especially in word level model where unseen data had "he" as the subject but generated data  was predicting "she" as subject. 




**Question:** What did you observe in the outputs of char LSTM model1 vs char LSTM model2 ?

**Answer:** 
* Due to extra LSTM layer, the second model captures longer range dependencies better than the first model.
* The extra layer and longer training time indicates that model has in depth understanding of corpus and performs better encoding and prediction. 
* On seen data predictions of char LSTM2 are almost perfect unlike that of char LSTM1 justifying the inference that second model has better understanding of training corpus.


# Bonus (Not to be graded)

## Transformer based language model (Bert)
You can run the below cells to predict the next word using pretrained BERT model. 

In [ ]:
!pip install transformers

In [ ]:
import os
import torch
import string
from transformers import BertTokenizer, BertForMaskedLM

In [ ]:
def load_model(model_name):
  try:
    if model_name.lower() == "bert":
      bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()
      return bert_tokenizer,bert_model
  except Exception as e:
    pass

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
  ignore_tokens = string.punctuation + '[PAD]'
  tokens = []
  for w in pred_idx:
    token = ''.join(tokenizer.decode(w).split())
    if token not in ignore_tokens:
      tokens.append(token.replace('##', ''))
  return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
  text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
  # if <mask> is the last token, append a "." so that models dont predict punctuation.
  if tokenizer.mask_token == text_sentence.split()[-1]:
    text_sentence += ' .'
  input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
  mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
  return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
  input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
  with torch.no_grad():
    predict = bert_model(input_ids)[0]
    print(predict.shape)
    
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
  return {'bert': bert}

In [ ]:
def get_prediction_eos(input_text):
  try:
    input_text += ' <mask>'
    res = get_all_predictions(input_text, top_clean=int(top_k))
    return res
  except Exception as error:
    pass

In [ ]:
# Below code predicts the next top_k words. You can modify this to get next n words using top_k=1 and greedy decoding it. 
top_k= 3
print('Predict next ', top_k, ' words')
model_name = 'BERT'
bert_tokenizer, bert_model  = load_model(model_name) 
input_text = "" ### GIVE YOUR INPUT STRING HERE
res = get_prediction_eos(input_text)
answer = []
print(res['bert'].split("\n"))
for i in res['bert'].split("\n"):
  answer.append(i)
  answer_as_string = "    ".join(answer)

print(answer_as_string)